### Demonstrate the LLM MPT Model OnBoarding on Cloud AI 100 Platform

##### Download the OpenSource MPT based HuggingFace Model and Save in local *Cache* directory
###### Now we Modify the MPT Classes using the Optimized Software Library to generate model for Cloud AI 100.
###### Here we generate models with below Optimizations:

* RMS Norm Fixes for FP16 Overflows and Underflow
* Causal Mask Fix
* Handling FP16 Overflows.
* KV Cache (Retention Changes).
* Triu/Tril Ops support.

In [ ]:
# Initiate the Original Transformer model

from QEfficient import QEFFAutoModelForCausalLM as AutoModelForCausalLM

# Please uncomment and use appropriate Cache Directory for transformers, in case you don't want to use default ~/.cache dir.
# os.environ["TRANSFORMERS_CACHE"] = "/local/mnt/workspace/hf_cache"

# ROOT_DIR = os.path.dirname(os.path.abspath(""))
# CACHE_DIR = os.path.join(ROOT_DIR, "tmp"), you can use a different location for just one model by passing this param as cache_dir in below API.

# Model-Card name to be onboarded (This is HF Model Card name) : https://huggingface.co/gpt2-xl
model_name = "mosaicml/mpt-7b"  # Similar, we can change model name and generate corresponding models, if we have added the support in the lib.
qeff_model = AutoModelForCausalLM.from_pretrained(model_name)
print(f"{model_name} optimized for Cloud AI 100 \n", qeff_model)

##### Export the Optimized Pytorch model to the Onnx Framework

In [ ]:
# We can now export the modified models to ONNX framework
# This will generate single ONNX Model for both Prefill and Decode Variations which are optimized for
# Cloud AI 100 Platform.

# While generating the ONNX model, this will clip the overflow constants to fp16
# Verify the model on Onnxruntime vs Pytorch

# Then generate inputs and customio yaml file required for compilation.
qeff_model.export()

##### Compile the Optimized KV Cache Single Model on Cloud AI 100 (**Config; 16C;32PL;128CTX;MXFP6**)

In [ ]:
# Compile the model for provided compilation arguments
# Please use platform SDK to Check num_cores for your card.

qeff_model.compile(num_cores=14, mxfp6=True)
# the device_group arg is deprecated from compile and added in generate method

##### Execute the Optimized KV Model on H/W and Print the Latency Stats *(tok/sec)*

In [ ]:
# post compilation, we can print the latency stats for the kv models, We provide API to print token and Latency stats on Cloud AI 100
# We need the compiled prefill and decode qpc to compute the token generated, This is based on Greedy Sampling Approach

qeff_model.generate(prompts=["My name is"], device_group=[0])